In [46]:
import base64

from cryptography.hazmat.primitives import serialization,hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed25519 import \
        Ed25519PublicKey, Ed25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend


In [47]:
from Cryptodome.Cipher import AES

In [48]:
def b64(msg):
    #base64 encoding helper function
    return base64.encodebytes(msg).decode('utf-8').strip()

In [49]:
def hkdf(inp,len):
    #using hashed message authenication code(HMAC) based key derivation function(KDF) to obtain the key from an input
    hkdf = HKDF(algorithm=hashes.SHA256(),length=len,salt=b'',info=b'',backend=default_backend())
    return hkdf.derive(inp)

In [50]:
class Bob(object):
    def __init__(self):
        #X25519 is an elliptical curve used to obtain cryptographic keys
        #IKb is the long term identity key of Bob
        self.IKb=X25519PrivateKey.generate()
        #SPKb is the signed pre key of Bob.The public key is uploaded to the server alongwith the signature using Bob's identity key,therefore proving Bob
        # has access to the private key
        self.SPKb=X25519PrivateKey.generate()
        #OPKb is the one time pre keys. Each ones public keys are published on the server.When wants to communicate with Bob it is fetched by him,then 
        #deleted from the server
        self.OPKb=X25519PrivateKey.generate()

    def x3dh(self,alice):
        #first DH key exchange between Bob's Signed pre key and Alice's Identity key.
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        #second DH key exchange between Bob's Identity Key and Alice's Ephemeral Key.
        #The first two DH key exchanges are for mutual authentication.
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        #third DH key exchange between Bob's One time pre key and Alice's ephemeral key.
        dh3 = self.OPKb.exchange(alice.EKa.public_key())
        #forth DH key exchange between Bob's Signed pre key and Alice's ephemeral key.
        dh4 = self.SPKb.exchange(alice.EKa.public_key())

        #Creation of the shared key by concatenating the four DH keys and applying HKDF on it
        self.sk=hkdf(dh1+dh2+dh3+dh4,32)
        print(f'[Bob] shared key: {b64(self.sk)}')

    def init_ratchet(self):
        self.root_ratchet = SymmRatchet(self.sk)
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

In [51]:
class Alice(object):
    #description of all are the same as the ones for Bob
    def __init__(self):
        self.IKa=X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()

    def x3dh(self,bob):
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.OPKb.public_key())
        dh4 = self.EKa.exchange(bob.SPKb.public_key())

        self.sk = hkdf(dh1+dh2+dh3+dh4,32)
        print(f'[Alice] shared key: {b64(self.sk)}')

    def init_ratchet(self):
        self.root_ratchet = SymmRatchet(self.sk)
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])

In [52]:
class SymmRatchet(object):
    def __init__(self,key):
        self.state = key

    def next(self,inp=b''):
        output = hkdf(self.state+inp,80)
        self.state = output[:32]
        outkey,iv=output[32:64],output[64:]
        return outkey,iv

In [53]:
# class Bob(object):

#     def init_ratchet(self):
#         self.root_ratchet = SymmRatchet(self.sk)
#         self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
#         self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

In [54]:
# class Alice(object):

#     def init_ratchet(self):
#         self.root_ratchet = SymmRatchet(self.sk)
#         self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
#         self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

In [55]:
alice = Alice()
bob = Bob()

In [56]:
alice.x3dh(bob)
bob.x3dh(alice)

alice.init_ratchet()
bob.init_ratchet()

print(f'[Alice] send ratchet: {list(map(b64,alice.send_ratchet.next()))}')
print(f'[Bob] recv ratchet: {list(map(b64,bob.recv_ratchet.next()))}')
print(f'[Bob] send ratchet: {list(map(b64,bob.send_ratchet.next()))}')
print(f'[Alice] recv ratchet: {list(map(b64,alice.recv_ratchet.next()))}')

[Alice] shared key: HKvgetcOY8zgCePilqPHR35acpiikxyhbFwUpk/R0xs=
[Bob] shared key: HKvgetcOY8zgCePilqPHR35acpiikxyhbFwUpk/R0xs=
[Alice] send ratchet: ['9sTDPRgOZunrO/U3D/9FOZfw4yysBwtMJbVxIM8bbR8=', 'vzNs4rTOvj7bEVm6J8clYw==']
[Bob] recv ratchet: ['9sTDPRgOZunrO/U3D/9FOZfw4yysBwtMJbVxIM8bbR8=', 'vzNs4rTOvj7bEVm6J8clYw==']
[Bob] send ratchet: ['ItLiLZpdmycouNkurRUVWFlKbo/X5hbj91mdj+uv83A=', 'edRwx6C2JL1Y/1Rd0U2ARg==']
[Alice] recv ratchet: ['ItLiLZpdmycouNkurRUVWFlKbo/X5hbj91mdj+uv83A=', 'edRwx6C2JL1Y/1Rd0U2ARg==']


In [65]:
print(f'[Alice] send ratchet: {list(map(b64,alice.send_ratchet.next()))}')
print(f'[Bob] recv ratchet: {list(map(b64,bob.recv_ratchet.next()))}')
print(f'[Bob] send ratchet: {list(map(b64,bob.send_ratchet.next()))}')
print(f'[Alice] recv ratchet: {list(map(b64,alice.recv_ratchet.next()))}')

[Alice] send ratchet: ['Jy7tBnKD5zYCi95GWV1DMjP+68qPkAxLYoq1EerN1uk=', 'al8dSpikVUaR6GgRGtd60g==']
[Bob] recv ratchet: ['Jy7tBnKD5zYCi95GWV1DMjP+68qPkAxLYoq1EerN1uk=', 'al8dSpikVUaR6GgRGtd60g==']
[Bob] send ratchet: ['uM/gsN/uuW/1nSQH8KyiCv//1bucEBZ4qtCI1UIAqRA=', '4k9dYhQuZqSnORhgSBnlrA==']
[Alice] recv ratchet: ['uM/gsN/uuW/1nSQH8KyiCv//1bucEBZ4qtCI1UIAqRA=', '4k9dYhQuZqSnORhgSBnlrA==']
